In [1]:
import sys
import os
import subprocess
from multiprocessing import Pool
from multiprocessing import cpu_count
import math

In [2]:
url = "./source.txt"

def list_urls(file):

    with open("./source.txt",'r') as fh:
        source_files_list = list(line.strip() for line in fh.readlines())
    return source_files_list
    
    
def download_dataset(url):
    try:
        subprocess.call(f"wget {url}",shell=True)
    except:
        pass

DATADIR = "/scratch1/rgoli/aws_customer_reviews"
def unzip_files(DATADIR):
    gz_files = os.listdir(DATADIR)
    source_files_list = list(line.strip() for line in gz_files)
    for file_ in source_files_list:
    #     print(file_)
        print(subprocess.call(["gzip","-d",os.path.join(DATADIR,file_)],shell=False))

In [3]:
source_files_list = list(filter(lambda file_:file_.endswith(".tsv"),os.listdir(DATADIR)))

In [4]:
import pyspark
print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_CONFIG_FILE'])
print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_MASTER_HOST'])
print(os.environ['SPARK_MASTER_PORT'])
print(os.environ['SPARK_MASTER_WEBUI_PORT'])

/software/spackages/linux-centos8-x86_64/gcc-8.3.1/spark-3.1.2-fbpdjvdj5fjlkjnu35rh4aooi2mirndz
/home/rgadde/ondemand/data/sys/dashboard/batch_connect/sys/palmetto_jupyter_spark/output/845c3df6-87ad-43ae-99f0-f0eba10dc72f/spark-defaults.conf
/software/spackages/linux-centos8-x86_64/gcc-8.3.1/spark-3.1.2-fbpdjvdj5fjlkjnu35rh4aooi2mirndz
node2092.palmetto.clemson.edu
28240
6669


In [5]:
import nltk
# nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words('english')
PUNCTUATIONS = """[!"#$%&()*+-.\/\\:;<=>?@\[\]^_`{|}\t\n\',~—]"""

In [6]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark import SparkContext
from pyspark.sql.types import StructType,StructField, StringType
import gzip
sc = SparkContext()

In [7]:
spark = pyspark.sql.SparkSession(sc)

In [8]:
from timeit import default_timer as timer

In [9]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import CountVectorizer
from timeit import default_timer as timer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, ArrayType


import time
import pandas as pd

In [10]:
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [11]:
import csv
def read_gunzip(path):
#     with gzip.open(path,'rt') as fh:
    data = spark.read.csv(path,sep="\t")
    return data
def read_tsv(path):
#     print(path)
    data = spark.read.csv(path,sep="\t",header = True)
    data = data.select(["review_body"])
    return data

In [12]:
data_files = list(filter(lambda x:x.endswith(".tsv"),os.listdir(DATADIR)))

In [13]:
review_objects = [file_.split("_")[3] for file_ in data_files]
data_files_parallelized = sc.parallelize([os.path.join(DATADIR,file_) for file_ in data_files])

In [14]:
data_files_parallelized_list = data_files_parallelized.collect()

In [15]:
read_t1 = timer()
dataframes = list(map(lambda file_:read_tsv(file_),data_files_parallelized_list))
read_t2 = timer()
print("Read time: ",read_t2 - read_t1)

Read time:  6.939342759549618


In [16]:
size = 0
for file_ in data_files:
    size += round(os.path.getsize(os.path.join(DATADIR,file_))*9.31*1e-10,2)
    print(round(size,2),int(int(size) % 12))

0.59 0
1.66 1
1.7 1
1.84 1
8.08 8
9.92 9
16.15 4
19.17 7
22.62 10
24.23 0
25.12 1
26.58 2
27.7 3
29.53 5
31.6 7
35.0 11
36.2 0
37.46 1
38.62 2
42.53 6
44.4 8
44.45 8
44.89 8
44.94 8
44.98 8
45.29 9
48.29 0
49.1 1
49.57 1
50.59 2
51.88 3
53.0 4
55.25 7
56.39 8
56.62 8
56.96 8
57.03 9
57.76 9
59.76 11
60.34 0
60.72 0
64.14 4
66.46 6
72.7 0
72.76 0
73.75 1


In [17]:
size *9.31*1e-10

6.866124999999998e-08

In [18]:
def tf_idf_pre_processing(dataframe,column_name):
    dataframe = dataframe.na.drop(subset=[column_name])
    dataframe = dataframe.withColumn(column_name,f.lower(f.col(column_name)))
    dataframe = dataframe.withColumn(column_name,f.regexp_replace(f.col(column_name), PUNCTUATIONS, ' '))
    dataframe = dataframe.withColumn(column_name,f.trim(f.col(column_name)))
    tokenizer = Tokenizer(inputCol=column_name, outputCol="words")
    dataframe = tokenizer.transform(dataframe)                  
    stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=STOPWORDS)
    data = stopwordsRemover.transform(dataframe)
    return data

def hash_tf(dataframe):
    starttime = timer()
#     tokenizer_hastingtf = Tokenizer(inputCol="review_body", outputCol="tokenized")
#     tokenized_data = tokenizer_hastingtf.transform(dataframe)
    hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
    result = hashingTF.transform(dataframe)
    print("Time to Hash", timer() - starttime)
    return result

def count_tf(dataframe):
    countVectors = CountVectorizer(inputCol="filtered", outputCol="rawFeatures", vocabSize=100000, minDF=5)
    model = countVectors.fit(dataframe)
    result = model.transform(dataframe)
    return result


def idf_generator(dataframe):
    starttime = timer()
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfmodel = idf.fit(dataframe)
    idf_data = idfmodel.transform(dataframe)
    print("Time to idf", timer() - starttime)
    return idf_data

In [19]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

# TF IDF with HasingTF and IDF

## Performance data size vs Computation Time

In [20]:
column_name = "review_body"
while True:
    size = 0
    time_dict = {}
    i=0
    for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        print(size,"\n")
        if int(int(size) % 12) == 0:
            starttime = timer()
            hashing_data = tf_idf_pre_processing(full_data_frame,column_name)
            result_hash = hash_tf(hashing_data)
            idf_data_hash = idf_generator(result_hash)
            time_to_idf = timer() - starttime
            time_dict.update({size:time_to_idf})
            print("Time to HashingTFIDF:", time_to_idf)
             
    break

0.59 

Time to Hash 0.037309326231479645
Time to idf 9.119319986552
Time to HashingTFIDF: 9.333043094724417
1.6600000000000001 

1.7000000000000002 

1.8400000000000003 

8.08 

9.92 

16.15 

19.169999999999998 

22.619999999999997 

24.229999999999997 

Time to Hash 0.010231640189886093
Time to idf 611.530782584101
Time to HashingTFIDF: 611.6154892109334
25.119999999999997 

26.58 

27.7 

29.53 

31.6 

35.0 

36.2 

Time to Hash 0.008726131170988083
Time to idf 874.8646063953638
Time to HashingTFIDF: 874.9420877359807
37.46 

38.62 

42.53 

44.4 

44.449999999999996 

44.88999999999999 

44.93999999999999 

44.97999999999999 

45.28999999999999 

48.28999999999999 

Time to Hash 0.007921997457742691
Time to idf 1136.538626063615
Time to HashingTFIDF: 1136.6134267523885
49.099999999999994 

49.56999999999999 

50.589999999999996 

51.879999999999995 

52.99999999999999 

55.24999999999999 

56.38999999999999 

56.61999999999999 

56.959999999999994 

57.029999999999994 

57.7599999

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/rgadde/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/rgadde/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2021.05-5tjen3mrle3pnguoedh4n2stzzsmywn7/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [24]:
metric_folder = "./metrics_1node"
with open(os.path.join(metric_folder,f"perf_results_hashtf_{48}.csv"),"a") as fh:
    fh.write("Size"+","+"Time"+"\n")
    for line in sorted(time_dict.keys()):
        fh.write(f"{line}"+","+f"{time_dict[line]}"+"\n")

## Performing time computation

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        if size > 24:
            break

In [ ]:
hashing_data = tf_idf_pre_processing(full_data_frame,column_name)
starttime = timer()
result_hash = hash_tf(hashing_data)
idf_data_hash = idf_generator(result_hash)
print("Time to HashingTFIDF:", timer() - starttime)

## TF IDF with CountVectorizer

In [ ]:
column_name = "review_body"
while True:
    size = 0
    time_dict_count = {}
    i=0
    for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        if int(int(size) % 12) == 0:
            starttime = timer()
            data = tf_idf_pre_processing(full_data_frame,column_name)
            result = count_tf(data)
            idf_data = idf_generator(result)
            time_to_idf = timer() - starttime
            time_dict_count.update({size:time_to_idf})
            print("Time to CountTFIDF:", time_to_idf)
        if i==5:
            break
        i+=1
    break
            

In [ ]:
metric_folder = "./metrics_1node"
with open(os.path.join(metric_folder,f"perf_results_counttf_{48}.csv"),"a") as fh:
    fh.write("Size"+","+"Time"+"\n")
    for line in sorted(time_dict_count.keys()):
        fh.write(f"{line}"+","+f"{time_dict_count[line]}"+"\n")

## Performing time computation

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        if size > 24:
            break

In [ ]:
hashing_data = tf_idf_pre_processing(full_data_frame,column_name)
data = tf_idf_pre_processing(full_data_frame,column_name)
result = count_tf(data)
idf_data = idf_generator(result)
print("Time to CountTFIDF:", timer() - starttime)